In [1]:
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import ipca
import my_functions_v13 as my
import IPCA_custom_module as ipca_custom
import importlib
import tqdm
from ipca import InstrumentedPCA
importlib.reload(my)
importlib.reload(ipca_custom)

<module 'IPCA_custom_module' from '/Users/lucasumann/PycharmProjects/Masterarbeit/Notebooks/IPCA_custom_module.py'>

In [2]:

number_factors = 5

In [3]:
data_model = full_coins_model = my.cond_auto_model(full_input = True, parallel_runs = 2, factor_dim=number_factors, model_name = "322 coins with intercept 14 provide data for IPCA")

In [4]:
data_model.initialize_X_Y()

In [5]:
X = [df.copy() for df in data_model.X[:-1]]
for df in X:
    del df["Intercept"]
Y = data_model.Y.copy()

mask = data_model.exclusion_mask.loc[Y.index].copy()


In [6]:
coin_indexer = {coin : number for coin, number in zip(Y.columns, np.arange(1, len(Y.columns) + 1))}

In [7]:
def transform_to_ipca_format(X, Y, mask):
    for df, coin in zip(X, Y.columns):
        df["day"] = np.arange(1, len(df) + 1)
        df["coin"] = coin_indexer[coin]
    X = pd.concat(X, axis=0, ignore_index=True)
    X.set_index(["coin", "day"], inplace=True)

    Y_indexer = {day: number for day, number in zip(Y.index, np.arange(1, len(Y) + 1))}
    Y.index = Y.index.map(Y_indexer)
    Y = Y.melt(ignore_index=False)
    Y = Y.rename(columns={"variable": "coin"})
    Y.loc[:, "coin"] = Y["coin"].map(coin_indexer)
    Y.set_index(["coin"], inplace=True, append=True)
    Y = Y.swaplevel()
    Y = Y.squeeze()

    mask_indexer = {day: number for day, number in zip(mask.index, np.arange(1, len(mask) + 1))}
    mask.index = mask.index.map(mask_indexer)
    mask = mask.melt(ignore_index=False)
    mask = mask.rename(columns={"variable": "coin"})
    mask.loc[:, "coin"] = mask["coin"].map(coin_indexer)
    mask.set_index(["coin"], inplace=True, append=True)
    mask = mask.swaplevel()
    mask = mask.squeeze()

    #remove unwanted
    X = X.loc[~mask]
    Y = Y.loc[~mask]

    X = X.swaplevel().sort_index().swaplevel()
    Y = Y.swaplevel().sort_index().swaplevel()

    return X, Y, mask


In [8]:
results = {}

In [9]:
for factor_number in tqdm.tqdm(np.arange(3, 5)):
    factors = int(factor_number)
    regr = ipca_custom.IPCA_custom(n_factors=factors, intercept=True)
    results[factors] = {}
    i = 0
    ongoing = True

    while ongoing:
        # Same window length as for training_percent = 0.3
        X_train = [df.iloc[: 360 + i * 30].copy() for df in X]
        Y_train = Y.iloc[: 360 + i * 30].copy(())
        X_test = [df.iloc[360 + i * 30: 360 + (i + 1) * 30].copy() for df in X]
        Y_test = Y.iloc[360 + i * 30: 360 + (i + 1) * 30].copy()

        testdays = len(Y_test)

        mask_train = mask.loc[Y_train.index].copy()
        mask_test = mask.loc[Y_test.index].copy()

        X_train, Y_train, mask_train = transform_to_ipca_format(X_train, Y_train, mask_train)
        X_test, Y_test, mask_test = transform_to_ipca_format(X_test, Y_test, mask_test)

        regr.fit(X=X_train, y=Y_train, data_type="panel")

        y_hat_predictive = regr.predict(X=X_test, mean_factor=True)

        results[factors][i] = {"y_hat_predictive": y_hat_predictive, "y_true_predictive": Y_test}

        y_hat_cont = pd.Series(np.full(0, np.nan))
        y_test_cont = pd.Series(np.full(0, np.nan))
        for j in np.arange(1, testdays + 1):
            X_index_j = [id for id in X_test.index if id[1] == j]
            Y_index_j = [id for id in Y_test.index if id[1] == j]

            X_test_j = X_test.loc[X_index_j]
            Y_test_j = Y_test.loc[Y_index_j]

            regr.updateFactorsOOS(X = X_test_j, y = Y_test_j)

            y_hat_cont_j = pd.Series(regr.predict(X_test_j))
            y_hat_cont_j.index = Y_test_j.index

            y_hat_cont = pd.concat([y_hat_cont, y_hat_cont_j])
            y_test_cont = pd.concat([y_test_cont, Y_test_j])

        results[factors][i]["y_hat_contemporaneous"] = y_hat_cont
        results[factors][i]["y_true_contemporaneous"] = y_test_cont

        i = i + 1

        ongoing = len(Y_test) >= 30

    sum_of_squared_residuals_predictive = 0
    sum_of_squares_predictive = 0
    for i in results[factors].keys():
        y_hat = results[factors][i]["y_hat_predictive"]
        y_test = results[factors][i]["y_true_predictive"]
        sum_of_squared_residuals_predictive += ((y_hat - y_test) ** 2).sum().sum()
        sum_of_squares_predictive += ((y_test) ** 2).sum().sum()

    sum_of_squared_residuals_total = 0
    sum_of_squares_total = 0
    for i in results[factors].keys():
        y_hat = results[factors][i]["y_hat_contemporaneous"]
        y_test = results[factors][i]["y_true_contemporaneous"]
        sum_of_squared_residuals_total += ((y_hat - y_test) ** 2).sum().sum()
        sum_of_squares_total += ((y_test) ** 2).sum().sum()


    results[factors]["predictive R squared"] = 1 - sum_of_squared_residuals_predictive / sum_of_squares_predictive
    results[factors]["total R squared"] = 1 - sum_of_squared_residuals_total / sum_of_squares_total


                              ]   0%[=                                                                       ]   2%[===                                                                     ]   4%[====                                                                    ]   6%[======                                                                  ]   8%[========                                                                ]  11%[=========                                                               ]  13%[===========                                                             ]  15%[============                                                            ]  17%[==============                                                          ]  20%[================                                                        ]  22%[=================                                                       ]  24%[===================                                                     ]  26%[===============

The panel dimensions are:
n_samples: 291 , L: 27 , T: 360


[========================================================================] 100%


Step 1 - Aggregate Update: 0.8154788770767885
Step 2 - Aggregate Update: 0.9560135127383325
Step 3 - Aggregate Update: 0.14153232692554618
Step 4 - Aggregate Update: 0.09112502174876097
Step 5 - Aggregate Update: 0.10843099764256489
Step 6 - Aggregate Update: 0.14477567427280713
Step 7 - Aggregate Update: 0.26490580576923717
Step 8 - Aggregate Update: 0.8468600125294363
Step 9 - Aggregate Update: 0.08201656537166888
Step 10 - Aggregate Update: 0.059282459065631135
Step 11 - Aggregate Update: 0.045801549080074166
Step 12 - Aggregate Update: 0.03654960411783456
Step 13 - Aggregate Update: 0.030139728495962637
Step 14 - Aggregate Update: 0.026901421364971506
Step 15 - Aggregate Update: 0.02418191537934683
Step 16 - Aggregate Update: 0.02179748220325095
Step 17 - Aggregate Update: 0.01964587485689774
Step 18 - Aggregate Update: 0.017673449163843724
Step 19 - Aggregate Update: 0.01585430142282493
Step 20 - Aggregate Update: 0.01417688368114223
Step 21 - Aggregate Update: 0.01263582469173034

The panel dimensions are:
n_samples: 297 , L: 27 , T: 390


...

Step 1 - Aggregate Update: 0.7778707609203909
Step 2 - Aggregate Update: 1.144090593984723
Step 3 - Aggregate Update: 0.11031428915133129
Step 4 - Aggregate Update: 0.08601934778256959
Step 5 - Aggregate Update: 0.056442008854782566
Step 6 - Aggregate Update: 0.04833725394003041
Step 7 - Aggregate Update: 0.04128618203358436
Step 8 - Aggregate Update: 0.03523444081607019
Step 9 - Aggregate Update: 0.030063908780354476
Step 10 - Aggregate Update: 0.025635964087937113
Step 11 - Aggregate Update: 0.02184823125319832
Step 12 - Aggregate Update: 0.018618688836538944
Step 13 - Aggregate Update: 0.015875411856307114
Step 14 - Aggregate Update: 0.01355309790508022
Step 15 - Aggregate Update: 0.011592389163409655
Step 16 - Aggregate Update: 0.009939980204961368
Step 17 - Aggregate Update: 0.008548745566373223
Step 18 - Aggregate Update: 0.007377651489949927
Step 19 - Aggregate Update: 0.0063914339054003955
Step 20 - Aggregate Update: 0.0055601053535461464
Step 21 - Aggregate Update: 0.004858369

The panel dimensions are:
n_samples: 322 , L: 27 , T: 1380


[========================================================================] 100%


Step 1 - Aggregate Update: 0.9386621824151158
Step 2 - Aggregate Update: 0.9204772442609214
Step 3 - Aggregate Update: 0.13268257453011958
Step 4 - Aggregate Update: 0.09143066973735836
Step 5 - Aggregate Update: 0.0596902962384116
Step 6 - Aggregate Update: 0.04323088662705954
Step 7 - Aggregate Update: 0.03199960878637875
Step 8 - Aggregate Update: 0.024100480472826924
Step 9 - Aggregate Update: 0.018383607973762484
Step 10 - Aggregate Update: 0.014180462323888629
Step 11 - Aggregate Update: 0.011061423906659748
Step 12 - Aggregate Update: 0.008730118161597772
Step 13 - Aggregate Update: 0.006975239526663721
Step 14 - Aggregate Update: 0.005644184210436637
Step 15 - Aggregate Update: 0.004626214559417537
Step 16 - Aggregate Update: 0.003840781022085453
Step 17 - Aggregate Update: 0.0032291507635924463
Step 18 - Aggregate Update: 0.0027483543704765953
Step 19 - Aggregate Update: 0.0024246393429347007
Step 20 - Aggregate Update: 0.002488879910680808
Step 21 - Aggregate Update: 0.002607

 50%|█████     | 1/2 [18:01:29<18:01:29, 64889.57s/it]

KeyboardInterrupt



In [ ]:
results[1]["predictive R squared"]

In [ ]:
results[1]["total R squared"]

In [ ]:
results_2 = {}

In [ ]:
for factor_number in tqdm.tqdm(np.arange(1, 2)):
    factors = int(factor_number)
    regr = ipca_custom.IPCA_custom(n_factors=factors, intercept=True)
    results_2[factors] = {}
    i = 0
    ongoing = True

    while ongoing:
        # Same window length as for training_percent = 0.3
        X_train = [df.iloc[: 512 + i * 30].copy() for df in X]
        Y_train = Y.iloc[: 512 + i * 30].copy(())
        X_test = [df.iloc[360 + i * 30: 512 + (i + 1) * 30].copy() for df in X]
        Y_test = Y.iloc[360 + i * 30: 512 + (i + 1) * 30].copy()

        testdays = len(Y_test)

        mask_train = mask.loc[Y_train.index].copy()
        mask_test = mask.loc[Y_test.index].copy()

        X_train, Y_train, mask_train = transform_to_ipca_format(X_train, Y_train, mask_train)
        X_test, Y_test, mask_test = transform_to_ipca_format(X_test, Y_test, mask_test)

        regr.fit(X=X_train, y=Y_train, data_type="panel")

        y_hat_predictive = regr.predict(X=X_test, mean_factor=True)

        results[factors][i] = {"y_hat_predictive": y_hat_predictive, "y_true_predictive": Y_test}

        y_hat_cont = pd.Series(np.full(0, np.nan))
        y_test_cont = pd.Series(np.full(0, np.nan))
        for j in np.arange(1, testdays + 1):
            X_index_j = [id for id in X_test.index if id[1] == j]
            Y_index_j = [id for id in Y_test.index if id[1] == j]

            X_test_j = X_test.loc[X_index_j]
            Y_test_j = Y_test.loc[Y_index_j]

            regr.updateFactorsOOS(X = X_test_j, y = Y_test_j)

            y_hat_cont_j = pd.Series(regr.predict(X_test_j))
            y_hat_cont_j.index = Y_test_j.index

            y_hat_cont = pd.concat([y_hat_cont, y_hat_cont_j])
            y_test_cont = pd.concat([y_test_cont, Y_test_j])

        results[factors][i]["y_hat_contemporaneous"] = y_hat_cont
        results[factors][i]["y_true_contemporaneous"] = y_test_cont

        i = i + 1

        ongoing = len(Y_test) >= 30

    sum_of_squared_residuals_predictive = 0
    sum_of_squares_predictive = 0
    for i in results[factors].keys():
        y_hat = results[factors][i]["y_hat_predictive"]
        y_test = results[factors][i]["y_true_predictive"]
        sum_of_squared_residuals_predictive += ((y_hat - y_test) ** 2).sum().sum()
        sum_of_squares_predictive += ((y_test) ** 2).sum().sum()

    sum_of_squared_residuals_total = 0
    sum_of_squares_total = 0
    for i in results[factors].keys():
        y_hat = results[factors][i]["y_hat_contemporaneous"]
        y_test = results[factors][i]["y_true_contemporaneous"]
        sum_of_squared_residuals_total += ((y_hat - y_test) ** 2).sum().sum()
        sum_of_squares_total += ((y_test) ** 2).sum().sum()


    results[factors]["predictive R squared"] = 1 - sum_of_squared_residuals_predictive / sum_of_squares_predictive
    results[factors]["total R squared"] = 1 - sum_of_squared_residuals_total / sum_of_squares_total
